![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&dt=Autoencoders+-+Data.ipynb)

# Autoencoders - Data

How to retrieve data for training, and using, an autoencoder.



---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20Autoencoders/Autoencoders%20-%20Data.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [43]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [44]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [45]:
# tuples of (import name, install name)
packages = [
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('google.cloud.bigquery_storage', 'google-cloud-bigquery-storage'),
    ('bigframes', 'bigframes'),
    ('pandas_gbq', 'pandas-gbq'),
    ('tensorflow', 'tensorflow', '2.10'),
    ('tensorflow_io', '--no-deps tensorflow-io'),
    ('graphviz', 'graphviz'),
    ('pydot', 'pydot')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

In [46]:
#!sudo apt-get -qq install graphviz

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [47]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [48]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [49]:
REGION = 'us-central1'
EXPERIMENT = 'data'
SERIES = 'applied-autoencoders'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# specify a GCS Bucket
GCS_BUCKET = PROJECT_ID

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id,splits' # add more variables to the string with comma delimiters

packages:

In [72]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import bigquery
from google.cloud import bigquery_storage
import bigframes.pandas as bpd
import pandas as pd
import concurrent.futures

from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf

#from datetime import datetime

#from google.protobuf import json_format
#from google.protobuf.struct_pb2 import Value
#import json
#import numpy as np


clients:

In [51]:
bq = bigquery.Client(project = PROJECT_ID)
bqstorage = bigquery_storage.BigQueryReadClient()
bpd.options.bigquery.project = PROJECT_ID

---
## DEV - Document later

### BigQuery Cell Magic

https://cloud.google.com/python/docs/reference/bigquery/latest/magics

In [14]:
%%bigquery bq_data_magic
SELECT *
FROM `statmike-mlops-349915.fraud.fraud_prepped`

Query is running:   0%|          |

Downloading:   0%|          |

In [15]:
bq_data_magic.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST
4,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,-0.176957,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0,0,c8a5b93a-1598-4689-80be-4f9f5df0b8ce,TEST


In [19]:
type(bq_data_magic)

pandas.core.frame.DataFrame

In [27]:
bq_data_magic.shape

(284807, 33)

### BigQuery Python Client

https://cloud.google.com/python/docs/reference/bigquery/latest

In [17]:
bq_data_client = bq.query(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`').to_dataframe()
bq_data_client.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST
4,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,-0.176957,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0,0,c8a5b93a-1598-4689-80be-4f9f5df0b8ce,TEST


In [18]:
type(bq_data_client)

pandas.core.frame.DataFrame

In [28]:
bq_data_client.shape

(284807, 33)

### BigQuery BigFrames Client

https://cloud.google.com/python/docs/reference/bigframes/latest

In [25]:
bq_data_bigframes = bpd.read_gbq(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
bq_data_bigframes.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,146737,-8.313253,7.353339,-6.784848,-1.647257,-3.151681,-1.036468,-3.176614,4.783091,2.525343,...,0.553924,-1.551447,0.668017,-0.047452,1.246589,0.71985,0.89,0,9c23ed6c-805e-4c54-af95-dfb8cb34446a,TRAIN
1,41342,-0.950509,1.005787,2.004522,3.180712,-0.721331,0.622745,-0.367445,0.690638,-0.831808,...,-0.019173,0.438937,-0.462187,0.328724,-0.041549,0.082586,39.52,0,f23f1435-eaf1-455c-9877-9e3b1db6473d,TRAIN
2,154083,-2.41823,-1.114104,0.0731,-1.827987,0.743047,5.017568,-2.471885,1.141392,0.812167,...,0.712716,0.746484,-0.70748,0.632214,-0.536598,0.014419,25.0,0,a147e336-81d1-47f2-87ef-206843430e00,TRAIN
3,137227,-1.125604,0.873506,2.20555,0.725739,-0.107164,1.59102,-1.093884,-2.480163,0.097936,...,-0.377725,-1.07084,0.37751,-0.257676,0.009876,-0.124529,54.0,0,2a7353de-8003-48a5-8ddf-327b42a5731d,TRAIN
4,52804,-0.336088,1.064353,2.150498,4.428434,0.463562,1.12541,0.579549,-0.118975,-1.284248,...,-0.095214,0.014986,-0.458112,0.291475,-0.00217,-0.228903,27.81,0,fd69d5aa-4c53-43a7-8ba9-32bbda68eab0,TRAIN


In [26]:
type(bq_data_bigframes)

bigframes.dataframe.DataFrame

In [29]:
bq_data_bigframes.shape

(284807, 33)

In [30]:
bq_data_bigframes = bq_data_bigframes.to_pandas()
type(bq_data_bigframes)

pandas.core.frame.DataFrame

In [31]:
bq_data_bigframes.shape

(284807, 33)

### BigQuery Storage Client

https://cloud.google.com/python/docs/reference/bigquerystorage/latest

In [52]:
read_session = bqstorage.create_read_session(
    request = dict(
        parent = f'projects/{PROJECT_ID}',
        read_session = dict(
            table = f"projects/{BQ_PROJECT}/datasets/{BQ_DATASET}/tables/{BQ_TABLE}",
            data_format = bigquery_storage.types.DataFormat.ARROW,
            #read_options = dict(
                #row_restriction = "Amount > 0",
            #    selected_fields = [f'V{n+1}' for n in range(28)]
            #)
        ),
        max_stream_count = 0
    )
)

In [54]:
len(read_session.streams)

1

In [56]:
def read_stream(stream):
    # setup a reader
    reader = bqstorage.read_rows(name = stream.name)
    # read rows from reader into a dataframe.  Note this is actually multiple operations - read and convert
    return reader.to_dataframe()


bq_data_storage = []
with concurrent.futures.ThreadPoolExecutor(max_workers = len(read_session.streams)) as executor:
    futures = {
        executor.submit(read_stream, stream): stream for stream in read_session.streams
    }
    for future in concurrent.futures.as_completed(futures):
        stream = futures[future]
        bq_data_storage.append(future.result())

In [57]:
len(bq_data_storage)

1

In [59]:
bq_data_storage[0].shape

(284807, 33)

In [60]:
bq_data_storage = pd.concat(bq_data_storage)
bq_data_storage.shape

(284807, 33)

In [61]:
type(bq_data_storage)

pandas.core.frame.DataFrame

In [62]:
bq_data_storage.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST
4,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,-0.176957,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0,0,c8a5b93a-1598-4689-80be-4f9f5df0b8ce,TEST


### Indirect BigQuery with `pandas-gbq`

When working with [Pandas](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) the methods above show the client returning data to pandas dataframes.  This section will show a pandas mudule, [pandas-gbq](https://pandas-gbq.readthedocs.io/en/latest/) that wraps the BigQuery client so that pandas can retrieve BigQuery data to dataframes.

References:
- [Comparison of BigQuery Client with pandas-gbq](https://cloud.google.com/bigquery/docs/pandas-gbq-migration)

In [35]:
bq_data_pandasgbq = pd.read_gbq(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`', project_id = PROJECT_ID)
bq_data_pandasgbq.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST
4,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,-0.176957,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0,0,c8a5b93a-1598-4689-80be-4f9f5df0b8ce,TEST


In [36]:
type(bq_data_pandasgbq)

pandas.core.frame.DataFrame

In [37]:
bq_data_pandasgbq.shape

(284807, 33)

### TensorFlow I/O

https://www.tensorflow.org/io

In [94]:
query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{BQ_TABLE}'"
schema = bq.query(query).to_dataframe()

In [95]:
schema.data_type.unique().tolist()

['INT64', 'FLOAT64', 'STRING']

In [96]:
types = {
    'FLOAT64' : dtypes.float64,
    'INT64' : dtypes.int64,
    'STRING' : dtypes.string
}

In [97]:
bq_data_tfio = BigQueryClient()
type(bq_data_tfio)

tensorflow_io.python.ops.bigquery_dataset_ops.BigQueryClient

In [98]:
bq_data_tfio = bq_data_tfio.read_session(
    parent = f"projects/{PROJECT_ID}",
    project_id = BQ_PROJECT,
    table_id = BQ_TABLE,
    dataset_id = BQ_DATASET,
    selected_fields = schema.column_name.tolist(),
    output_types = [types[x] for x in schema.data_type.tolist()],
    #row_restriction = f"splits='{split}'",
    requested_streams = 3
)
type(bq_data_tfio)

tensorflow_io.python.ops.bigquery_dataset_ops.BigQueryReadSession

In [99]:
bq_data_tfio = bq_data_tfio.parallel_read_rows(sloppy = True, num_parallel_calls = tf.data.experimental.AUTOTUNE)
type(bq_data_tfio)

tensorflow.python.data.ops.interleave_op._ParallelInterleaveDataset

In [100]:
bq_data_tfio = bq_data_tfio.batch(5)
type(bq_data_tfio)

tensorflow.python.data.ops.batch_op._BatchDataset

In [101]:
for rows in bq_data_tfio.take(1):
    print(rows.keys())
    for item in rows.items():
        print(item)

odict_keys(['Amount', 'Class', 'Time', 'V1', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V2', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'splits', 'transaction_id'])
('Amount', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0., 0., 0., 0., 0.])>)
('Class', <tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>)
('Time', <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 35337,  60481, 139587, 162908, 165236])>)
('V1', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 1.09284419,  1.23897318,  1.87053898, -3.36833924,  2.1801491 ])>)
('V10', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 0.5874381 , -0.10984728,  1.40743985,  0.47968371,  0.28015212])>)
('V11', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([-0.14979756, -0.60471137, -1.54563924, -0.78039714, -1.15788317])>)
('V12', <tf.Tensor: shape=(5,), dtype=float64, numpy=array([ 0.29514782, -0.45648726, -0.7821938